In [2]:
import pandas as pd
import numpy as np

missing_token = " UNK "
train = pd.read_csv("Data/train.csv", parse_dates=["project_submitted_datetime"])
test = pd.read_csv("Data/test.csv", parse_dates=["project_submitted_datetime"])
ul = pd.read_csv("Data/Ultimate.csv")
#rc = pd.read_csv("Data/resources.csv").fillna(missing_token)
#nlp = pd.read_csv("Data/df_NLPed.csv")
#nlp =nlp.drop_duplicates('id')

In [41]:
def preprocess1(string):
    '''
    :param string:
    :return:
    '''
    #去掉一些特殊符号
    string = re.sub(r'(\r\n)', ' ', string)
    string = re.sub(r'(\r)', ' ', string)
    string = re.sub(r'(\n)', ' ', string)
    string = re.sub(r'(\\)', ' ', string)
    string = re.sub(r'\t', ' ', string)
    string = re.sub(r'\:', ' ', string)
    string = re.sub(r'\+', ' ', string)
    string = re.sub(r'\=', ' ', string)
    string = re.sub(r'(\")', ' ', string)
    string = re.sub(r'\"\"\"\"', ' ', string)
    string = re.sub(r'_', ' ', string)


    return string

In [46]:
pd.options.display.max_columns = 100
nlp.head()

,id,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_grade_category,project_is_approved,project_resource_summary,project_subject_categories,project_subject_subcategories,project_submitted_datetime,project_title,school_state,teacher_id,teacher_number_of_previously_posted_projects,teacher_prefix,items,quantity,price,total_price,min_quantity,min_total_price,max_quantity,max_price,max_total_price,mean_quantity,mean_price,mean_total_price,std_quantity,std_price,std_total_price,resource_description,Year,Month,Weekday,Hour,Month_Day,Year_Day,essay1_len,essay2_len,essay3_len,essay4_len,title_len,resource_summary_len,resource_description_len,resource_description_wc,title_wc,essay1_wc,essay2_wc,essay3_wc,essay4_wc,resource_summary_wc,text,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,stopword_count,polarity,subjectivity,noun_count,verb_count,adj_count,adv_count,pron_count,min_price
0,p036502,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,UNK,UNK,Grades PreK-2,1.0,My students need 6 Ipod Nano's to create and d...,Literacy & Language,Literacy,2016-11-18 14:45:59,Super Sight Word Centers,NV,484aaf11257089a66cfedc9461c6bd0a,26.0,Ms.,2.0,6.0,299.98,899.94,3.0,449.97,3.0,149.99,449.97,3.0,149.990,449.970,0.0,0.000000,0.000000,Apple - iPod nano� 16GB MP3 Player (8th Genera...,2016.0,11.0,4.0,14.0,18.0,323.0,967.0,805.0,5.0,5.0,24.0,127.0,149.0,28.0,4.0,170.0,142.0,3.0,3.0,21.0,Most of my kindergarten students come from low...,1785.0,314.0,5.666667,40.0,21.0,7.0,151.0,0.213402,0.391136,81.0,58.0,25.0,16.0,36.0,149.99
1,p039565,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,UNK,UNK,Grades 3-5,0.0,My students need matching shirts to wear for d...,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",2017-04-26 15:57:28,Keep Calm and Dance On,GA,df72a3ba8089423fa8a94be88060f6ed,1.0,Mrs.,1.0,20.0,20.00,400.00,20.0,400.00,20.0,20.00,400.00,20.0,20.000,400.000,0.0,0.000000,0.000000,Reebok Girls' Fashion Dance Graphic T-Shirt - ...,2017.0,4.0,2.0,15.0,26.0,116.0,587.0,639.0,5.0,5.0,22.0,81.0,70.0,13.0,5.0,94.0,102.0,3.0,3.0,12.0,Our elementary school is a culturally rich sch...,1239.0,192.0,6.419689,38.0,15.0,5.0,79.0,0.192889,0.597111,59.0,27.0,25.0,10.0,19.0,20.00
2,p233823,Hello; r nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,UNK,UNK,Grades 3-5,1.0,My students need the 3doodler. We are an SEM s...,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",2017-01-01 22:57:44,Lets 3Doodle to Learn,UT,a9b876a9252e08a55e3d894150f75ba3,5.0,Ms.,1.0,1.0,469.99,469.99,1.0,469.99,1.0,469.99,469.99,1.0,469.990,469.990,0.0,0.000000,0.000000,3doodler Start Full Edu Bundle,2017.0,1.0,6.0,22.0,1.0,1.0,761.0,546.0,5.0,5.0,21.0,186.0,30.0,5.0,4.0,135.0,101.0,3.0,3.0,33.0,Hello; r nMy name is Mrs. Brotherton. I teach ...,1320.0,236.0,5.569620,26.0,26.0,6.0,103.0,0.353888,0.534450,58.0,51.0,19.0,7.0,32.0,469.99
3,p185307,My students are the greatest students but are ...,The student's project which is totally kid-i...,UNK,UNK,Grades 3-5,0.0,My students need balls and other activity equi...,Health & Sports,Health & Wellness,2016-08-12 15:42:11,Kid Inspired Equipment to Increase Activit...,NC,525fdbb6ec7f538a48beebaa0a51b24f,16.0,Mr.,5.0,5.0,684.47,684.47,1.0,18.95,1.0,354.99,354.99,1.0,136.894,136.894,0.0,133.428098,133.428098,BALL PG 4'' POLY SET OF 6 COLORS BALL PLAYGROU...,2016.0,8.0,4.0,15.0,12.0,225.0,1201.0,1209.0,5.0,5.0,72.0,233.0,148.0,27.0,10.0,206.0,202.0,3.0,3.0,37.0,My students are the greatest students but are ...,2423.0,388.0,6.228792,77.0,31.0,6.0,188.0,0.175880,0.416224,105.0,78.0,37.0,23.0,34.0,18.95
4,p013780,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,UNK,UNK,Grades 6-8,1.0,My students need a water filtration system for...,Health & Sports,Heal

In [42]:
cols = ['project_essay_1','project_essay_2','project_essay_3','project_essay_4',
        'project_resource_summary','project_title','resource_description','text']

In [45]:
import re
for c in cols:
    nlp[c] = nlp[c].astype(str)
    nlp[c] = nlp[c].apply(preprocess1)

In [11]:
df = nlp.copy()

In [21]:
%%time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
# Text Features for feature extraction
text_cols = ["text","project_resource_summary", "project_title", "resource_description"]

# Sentiment Build
print("Hand Made Text Features..")
SIA = SentimentIntensityAnalyzer()
for cols in tqdm(text_cols):
    df[cols] = df[cols].astype(str) # Make Sure data is treated as string
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols+'_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols+'_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols+'_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    # Count Unique Words
    df[cols+'_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words']*100
    # Unique words to Word count Ratio
    if cols == "text":
        df[cols+"_vader_Compound"]= df[cols].apply(lambda x:SIA.polarity_scores(x)['compound'])
        df[cols+'_vader_Neutral']= df[cols].apply(lambda x:SIA.polarity_scores(x)['neu'])
        df[cols+'_vader_Negative']= df[cols].apply(lambda x:SIA.polarity_scores(x)['neg'])
        df[cols+'_vader_Positive']= df[cols].apply(lambda x:SIA.polarity_scores(x)['pos'])
        # Test a Stemmer..
    print("{} Done".format(cols))


  0%|          | 0/4 [00:00<?, ?it/s]

Hand Made Text Features..



Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/healer/src/tqdm/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 25%|██▌       | 1/4 [1:15:31<3:46:34, 4531.55s/it]

text Done


 50%|█████     | 2/4 [1:15:40<1:15:40, 2270.30s/it]

project_resource_summary Done


 75%|███████▌  | 3/4 [1:15:44<25:14, 1514.96s/it]  

project_title Done


100%|██████████| 4/4 [1:16:05<00:00, 1141.34s/it]

resource_description Done
CPU times: user 1h 36s, sys: 1min 25s, total: 1h 2min 1s
Wall time: 1h 16min 5s


In [22]:
df.to_csv('Data/New_NLP.csv',index = False)

In [23]:
tfidf_para = {
    "sublinear_tf":True,
    "strip_accents":'unicode',
    "stop_words":"english",
    "analyzer":'word',
    "token_pattern":r'\w{1,}',
    #"ngram_range":(1,1),
    "dtype":np.float32,
    "norm":'l2',
    "min_df":5,
    "max_df":.9,
    "smooth_idf":False
}

In [30]:
# Thanks To
# https://www.kaggle.com/lopuhin/eli5-for-mercari
# https://www.kaggle.com/jagangupta/understanding-approval-donorschoose-eda-fe-eli5/notebook
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_col(col_name):
    return lambda x: x[col_name]

df["project_title_count"] = df["project_title"].copy()
textcols = ["text","project_resource_summary","project_title", "project_title_count","resource_description"]
vectorizer = FeatureUnion([
        ('text',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=20000,
            **tfidf_para,
            preprocessor=get_col('text'))),
        ('project_resource_summary',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=2000,
            preprocessor=get_col('project_resource_summary'))),
        ('project_title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=1500,
            preprocessor=get_col('project_title'))),
        ('project_title_count',CountVectorizer(
            ngram_range=(1, 2),
            max_features=1500,
            preprocessor=get_col('project_title_count'))),
        ('resource_description',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=2400,
            preprocessor=get_col('resource_description'))),
#         ('Non_text',DictVectorizer())
    ])

In [31]:
import time
start_vect=time.time()
ready_df = vectorizer.fit_transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

Vectorization Runtime: 7.18 Minutes


In [34]:
df.shape

(317920, 89)

In [40]:
ready_df[0:10]

<10x27400 sparse matrix of type '<class 'numpy.float64'>'
	with 1660 stored elements in Compressed Sparse Row format>

In [36]:
df.head()

,id,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_grade_category,project_is_approved,project_resource_summary,project_subject_categories,project_subject_subcategories,project_submitted_datetime,project_title,school_state,teacher_id,teacher_number_of_previously_posted_projects,teacher_prefix,items,quantity,price,total_price,min_quantity,min_total_price,max_quantity,max_price,max_total_price,mean_quantity,mean_price,mean_total_price,std_quantity,std_price,std_total_price,resource_description,Year,Month,Weekday,Hour,Month_Day,Year_Day,essay1_len,essay2_len,essay3_len,essay4_len,title_len,resource_summary_len,resource_description_len,resource_description_wc,title_wc,essay1_wc,essay2_wc,essay3_wc,essay4_wc,resource_summary_wc,text,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,stopword_count,polarity,subjectivity,noun_count,verb_count,adj_count,adv_count,pron_count,min_price,text_num_chars,text_num_words,text_num_unique_words,text_words_vs_unique,text_vader_Compound,text_vader_Neutral,text_vader_Negative,text_vader_Positive,project_resource_summary_num_chars,project_resource_summary_num_words,project_resource_summary_num_unique_words,project_resource_summary_words_vs_unique,project_title_num_chars,project_title_num_words,project_title_num_unique_words,project_title_words_vs_unique,resource_description_num_chars,resource_description_num_words,resource_description_num_unique_words,resource_description_words_vs_unique,project_title_count
0,p036502,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,UNK,UNK,Grades PreK-2,1.0,my students need 6 ipod nano's to create and d...,Literacy & Language,Literacy,2016-11-18 14:45:59,super sight word centers,NV,484aaf11257089a66cfedc9461c6bd0a,26.0,Ms.,2.0,6.0,299.98,899.94,3.0,449.97,3.0,149.99,449.97,3.0,149.990,449.970,0.0,0.000000,0.000000,apple - ipod nano� 16gb mp3 player (8th genera...,2016.0,11.0,4.0,14.0,18.0,323.0,967.0,805.0,5.0,5.0,24.0,127.0,149.0,28.0,4.0,170.0,142.0,3.0,3.0,21.0,most of my kindergarten students come from low...,1785.0,314.0,5.666667,40.0,21.0,7.0,151.0,0.213402,0.391136,81.0,58.0,25.0,16.0,36.0,149.99,1785,321,174,54.205607,0.9683,0.882,0.021,0.097,127,21,19,90.476190,24,4,4,100.0,149,28,13,46.428571,super sight word centers
1,p039565,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,UNK,UNK,Grades 3-5,0.0,my students need matching shirts to wear for d...,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",2017-04-26 15:57:28,keep calm and dance on,GA,df72a3ba8089423fa8a94be88060f6ed,1.0,Mrs.,1.0,20.0,20.00,400.00,20.0,400.00,20.0,20.00,400.00,20.0,20.000,400.000,0.0,0.000000,0.000000,reebok girls' fashion dance graphic t-shirt - ...,2017.0,4.0,2.0,15.0,26.0,116.0,587.0,639.0,5.0,5.0,22.0,81.0,70.0,13.0,5.0,94.0,102.0,3.0,3.0,12.0,our elementary school is a culturally rich sch...,1239.0,192.0,6.419689,38.0,15.0,5.0,79.0,0.192889,0.597111,59.0,27.0,25.0,10.0,19.0,20.00,1239,202,137,67.821782,0.9052,0.884,0.022,0.095,81,12,12,100.000000,22,5,5,100.0,70,13,12,92.307692,keep calm and dance on
2,p233823,Hello; r nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,UNK,UNK,Grades 3-5,1.0,my students need the 3doodler. we are an sem s...,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",2017-01-01 22:57:44,lets 3doodle to learn,UT,a9b876a9252e08a55e3d894150f75ba3,5.0,Ms.,1.0,1.0,469.99,469.99,1.0,469.99,1.0,469.99,469.99,1.0,469.990,469.990,0.0,0.000000,0.000000,3doodler start full edu bundle,2017.0,1.0,6.0,22.0,1.0,1.0,761.0,546.0,5.0,5.0,21.0,186.0,30.0,5.0,4.0,135.0,101.0,3.0,3.0,33.0,hello; r nmy name is mrs. brotherton. i teach ...,1320.0,236.0,5.569620,26.0,26.0,6.0,103.0,0.353888,0.534450,58.0,51.0,19.0,7.0,32.0,469.99,1320,241,137,56.846473,0.9927,0.787,0.011,0.203,186,33,29,87.878788,21,4,4,100.0,30,5,5,100.000000,let